In [230]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
import time

In [371]:
# Part (a)
#equations were 
#     KCL:
#     I0=I1+I2
#     I1+I5=I7
#     I2=I3+I4 (ignoring this one as there are only 8 unknows)
#     I3=I5+I6
#     I4+I6+I7=I0 
#     KVL:
#     20-2I0-I2-2I4=0  
#     -2I1+2I5+2I3+I2=0
#     +2I3+2I6-I4=0
#     +2I5+2I1-2I6=0

A = np.array([
    [1., -1., -1., 0., 0., 0., 0., 0.],  # KCL  tick
    [0., 1., 0., 0., 0., 1., 0., -1.],  # KCL tick 
    [0., 0., 0., 1., 0., -1., -1., 0.], # KCL tick
    [0,  0,  1, -1, -1,  0,  0,  0], #KCL tick
    [2., 0., 1., 0., 2., 0., 0., 0.],   # KVL
    [0., -2., 1., 2., 0., 2., 0., 0.],  # KVL 
    [0., 0., 0., 2., -1., 0., 2., 0.],  # KVL 
    [0., 2., 0., 0., 0., 2., -2., 0.]   # KVL
])
#checking singularity
det_A = np.linalg.det(A)
print(f"Determinant of matrix A: {det_A}")
b = np.array([0., 0., 0., 0., 20.,0., 0., 0.])



Determinant of matrix A: -340.0000000000001


In [385]:
# Part (b)
def rowSwap(v, i, j):
    if len(v.shape) == 1:
        v[i], v[j] = v[j], v[i]
    else:
        v[[i, j], :] = v[[j, i], :]

def LUpivot(A, tol=1.0e-15):
    n = len(A)
    perm = np.array(range(n))
    for k in range(0, n-1):
        p = np.argmax(np.abs(A[k:n, k])) + k
        if np.abs(A[p, k]) < tol:
            raise ValueError("Singular matrix detected")
        if p != k:
            rowSwap(A, k, p)
            rowSwap(perm, k, p)
        for i in range(k+1, n):
            if A[i, k] != 0.0:
                mik = A[i, k] / A[k, k]
                A[i, k+1:n] -= mik * A[k, k+1:n]
                A[i, k] = mik
    return A, perm

def solveLUpivot(A, b, perm):
    n = len(A)
    x = b.copy()
    for i in range(n):
        x[i] = b[perm[i]]
    for k in range(1, n):
        x[k] -= np.dot(A[k, 0:k], x[0:k])
    x[n-1] /= A[n-1, n-1]
    for k in range(n-2, -1, -1):
        x[k] = (x[k] - np.dot(A[k, k+1:n], x[k+1:n])) / A[k, k]
    return x
start_lu = time.time()
A_factored, perm = LUpivot(np.copy(A))  
x = solveLUpivot(A_factored, b, perm)
end_lu = time.time()
print("Solution x:", x)
print(f"Time taken (LU with pivoting): {end_lu - start_lu} seconds\n")

verification_lu = np.copy(A) @ x 
verification_difference_lu = np.abs(verification_lu - b)  

print("Verification (A · x):", verification_lu)
print("Original RHS vector (b):", b)
print("Difference (should be near zero):", verification_difference_lu)



Solution x: [ 5.76583291  1.35574718  7.97087578 -4.59639219 -9.7512708   1.96670148
 -0.27924321 -2.76730996]
Time taken (LU with pivoting): 0.0009984970092773438 seconds

Verification (A · x): [ 8.88178420e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.00000000e+01 -1.77635684e-15  0.00000000e+00 -8.88178420e-16]
Original RHS vector (b): [ 0.  0.  0.  0. 20.  0.  0.  0.]
Difference (should be near zero): [8.88178420e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.77635684e-15 0.00000000e+00 8.88178420e-16]


In [395]:
# Part (c)
from scipy.sparse import csr_matrix
A_sparse = csr_matrix(A)

def gauss_seidel_sparse(A, b, x, tol=1.e-16, maxit=100):
    n = len(b)
    err = 1.0
    iters = 0
    xnew = np.zeros_like(x)
    while err > tol and iters < maxit:
        iters += 1
        for i in range(n):
            row_start = A.indptr[i]
            row_end = A.indptr[i + 1]
            row_indices = A.indices[row_start:row_end]
            row_data = A.data[row_start:row_end]

            sigma = sum(row_data[j] * xnew[row_indices[j]] for j in range(len(row_data)) if row_indices[j] != i)
            xnew[i] = (b[i] - sigma) / A[i, i]
        err = np.linalg.norm(xnew - x, np.inf)
        x = np.copy(xnew)
    print('Iterations required for convergence (sparse):', iters)
    return x

x0 = np.zeros_like(b)
start_gs = time.time()
A_sparse = csr_matrix(np.copy(A))  
x_gs_sparse = gauss_seidel_sparse(A_sparse, b, x0)
end_gs = time.time()
print("Gauss-Seidel (sparse) solution:", x_gs_sparse)
print(f"Time taken (Gauss-Seidel): {end_gs - start_gs} seconds\n")


verification_gs = np.copy(A) @ x_gs_sparse  
verification_difference_gs = np.abs(verification_gs - b)

print("Verification (A · x_gs):", verification_gs)
print("Original RHS vector (b):", b)
print("Difference (should be near zero):", verification_difference_gs)

tolerance = 1e-6
max_error = np.max(verification_difference_gs)
if max_error <= tolerance:
    print(f"Solution satisfies the tolerance: {max_error}")
else:
    print(f"Solution exceeds tolerance with a maximum error of {max_error}")



Iterations required for convergence (sparse): 100
Gauss-Seidel (sparse) solution: [ 5.76583291  1.35574718  7.97087578 -4.59639219 -9.7512708   1.96670148
 -0.27924321 -2.76730996]
Time taken (Gauss-Seidel): 0.017001867294311523 seconds

Verification (A · x_gs): [ 4.44089210e-15 -8.88178420e-16 -1.77635684e-15 -3.55271368e-15
  2.00000000e+01  8.88178420e-16  0.00000000e+00  0.00000000e+00]
Original RHS vector (b): [ 0.  0.  0.  0. 20.  0.  0.  0.]
Difference (should be near zero): [4.44089210e-15 8.88178420e-16 1.77635684e-15 3.55271368e-15
 0.00000000e+00 8.88178420e-16 0.00000000e+00 0.00000000e+00]
Solution satisfies the tolerance: 4.440892098500626e-15


In [403]:
# Part (d)
speedup = (end_lu - start_lu) / (end_gs - start_gs)
print(f"Speedup factor (dense/sparse): {speedup}\n")
speedup = (end_lu - start_lu) / (end_gs - start_gs)
print(f"Speedup factor (dense/sparse): {speedup}\n")
if speedup > 1:
    print("LU Decomposition (dense method) is faster.")
else:
    print("Gauss-Seidel (sparse method) is faster.")


Speedup factor (dense/sparse): 0.05872866738651821

Speedup factor (dense/sparse): 0.05872866738651821

Gauss-Seidel (sparse method) is faster.


In [405]:
# Part (e)
tolerance = 1e-6

difference = np.abs(x - x_gs_sparse)
print("Difference between dense (LU) and sparse (Gauss-Seidel) solutions:")
print(difference)
max_difference = np.max(difference)
print(f"Maximum difference between solutions: {max_difference}")
if max_difference <= tolerance:
    print(f"The solutions from dense and sparse methods are consistent within the tolerance of {tolerance}.")
else:
    print(f"The solutions from dense and sparse methods differ beyond the tolerance of {tolerance}. Maximum difference: {max_difference}")
error_lu = np.abs(A @ x - b)
error_gs = np.abs(A @ x_gs_sparse - b)  # 
print("LU Decomposition error (max):", np.max(error_lu))
print("Gauss-Seidel error (max):", np.max(error_gs))

if np.max(error_lu) < np.max(error_gs):
    print("LU Decomposition provides a more accurate solution.")
else:
    print("Gauss-Seidel provides a more accurate solution.")

Difference between dense (LU) and sparse (Gauss-Seidel) solutions:
[8.88178420e-16 1.99840144e-15 1.77635684e-15 8.88178420e-16
 0.00000000e+00 1.55431223e-15 3.33066907e-16 1.77635684e-15]
Maximum difference between solutions: 1.9984014443252818e-15
The solutions from dense and sparse methods are consistent within the tolerance of 1e-06.
LU Decomposition error (max): 1.7763568394002505e-15
Gauss-Seidel error (max): 4.440892098500626e-15
LU Decomposition provides a more accurate solution.
